In [1]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS.sort(key=len)

In [2]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
            columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [3]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
    df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].add_suffix('_run' + str(i)))
    mini_df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

In [4]:
get_summary_df(merged)

,usingfulltestset_3d1000_mean,usingfulltestset_3d1000_std,3d1000_mean,3d1000_std,usingfulltestset_3d560_mean,usingfulltestset_3d560_std,3d560_mean,3d560_std
0,79.420539,0.190451,70.00502,1.06271,58.726481,0.326393,57.696308,1.133374


In [5]:
mini_df_lst[0]

,3d560_run0,usingfulltestset_3d560_run0,3d1000_run0,usingfulltestset_3d1000_run0
0,56.694145,58.295676,69.142736,79.418923


In [10]:
mini_df_lst[1]

,3d560_run1,usingfulltestset_3d560_run1,3d1000_run1,usingfulltestset_3d1000_run1
0,58.41466,58.655095,71.054022,79.367814


In [12]:
mini_df_lst[2]

,3d560_run2,usingfulltestset_3d560_run2,3d1000_run2,usingfulltestset_3d1000_run2
0,56.867297,59.072343,69.729168,79.742007


In [13]:
mini_df_lst[3]

,3d560_run3,usingfulltestset_3d560_run3,3d1000_run3,usingfulltestset_3d1000_run3
0,59.327742,59.032242,71.188315,79.24646


In [14]:
mini_df_lst[4]

,3d560_run4,usingfulltestset_3d560_run4,3d1000_run4,usingfulltestset_3d1000_run4
0,57.177697,58.57705,68.91086,79.32749
